# RNA-seq Pipeline

## Be sure to install paramiko and scp with pip before using this notebook

## 1. Configure AWS key pair, data location on S3 and the project information

This cell only contains information that you, the user, should input.

#### String Fields

**s3_input_files_address**: This is an s3 path to where your input fastq files are found. This shouldn't be the path to the actual fastq files, just to the directory containing all of them. All fastq files must be in the same s3 bucket.

**s3_output_files_address**: This is an s3 path to where you would like the outputs from your project to be uploaded. This will only be the root directory, please see the README for information about exactly how outputs are structured

**design_file**: This is a path to your design file for this project. Please see the README for the format specification for the design files. 

**your_cluster_name**: This is the name given to your cluster when it was created using cfncluster. 

**private_key**: The path to your private key needed to access your cluster.

**project_name**: Name of your project. There should be no whitespace.

**workflow**: The workflow you want to run for this project. For the RNASeq pipeline the possible workflows are "star_rsem", "star_gatk", "star_htseq", and "kallisto". 

**genome**: The name of the reference you want to use for your project. Currently only "hg19" is supported here.

#### analysis_steps
This is a set of strings that contains the steps you would like to run. The order of the steps does not matter.

posible star_gatk steps: "fastqc", "trim", "align", "multiqc", "variant_calling"

possible star_htseq steps: "fastqc", "trim", "align", "multiqc", "count", "merge_counts"

possible star_rsem steps: "fastqc", "trim", "align_count", "multiqc", "merge_counts"

possible kallisto steps: "fastqc", "trim", "align", "multiqc", "count", "merge_counts"

In [ ]:
import os
import sys
from cirrusngs.managers import ConnectionManager, PipelineManager, CFNClusterManager
from cirrusngs.util import DesignFileLoader

s3_input_files_address = "{s3_input_files_address}"
s3_output_files_address = "{s3_output_files_address}"

design_file = "{design_file}"

your_cluster_name = "{your_cluster_name}"

private_key = "{private_key}"

project_name = "{project_name}"

workflow = "{workflow}"

genome = "{genome}"

## Analysis steps:
# for star_gatk: "fastqc", "trim", "align", "multiqc", "variant_calling"
# for star_htseq: "fastqc", "trim", "align", "multiqc", "count", "merge_counts"
# for star_rsem: "fastqc", "trim", "align_count", "multiqc", "merge_counts"
# for kallisto: "fastqc", "trim", "align", "multiqc", "count", "merge_counts"
analysis_steps = {
                    "fastqc"
                    ,"trim"
                    ,"align"
#                     ,"align_count"
                    ,"multiqc"
                    ,"count"
                    ,"merge_counts"
                }

## 2. Create CFNCluster

Following cell connects to your cluster. Run before step 3.

In [ ]:
## Create a new cluster
master_ip_address = CFNClusterManager.create_cfn_cluster(cluster_name=your_cluster_name)
ssh_client = ConnectionManager.connect_master(hostname=master_ip_address,
                                              username="ec2-user",
                                              private_key_file=private_key)

## 3. Run the RNA sequencing pipeline

This cell actually executes your pipeline. Make sure that steps 1 and 2 have been completed before running.

In [ ]:
sample_list, group_list, pairs_list = DesignFileLoader.load_design_file(design_file)

PipelineManager.execute("RNASeq", ssh_client, project_name, workflow, analysis_steps, s3_input_files_address,
                        sample_list, group_list, s3_output_files_address, genome, "NA", pair_list)

## 4. Check status of pipeline

This allows you to check the status of your pipeline. You can specify a step or set the step variable to "all". If you specify a step it should be one that is in your analysis_steps set. You can toggle how verbose the status checking is by setting the verbose flag (at the end of the second line) to False. 

In [ ]:
step="all"
PipelineManager.check_status(ssh_client, step, "RNASeq", workflow, project_name, analysis_steps,verbose=True)

If your pipeline is finished run this cell just in case there's some processes still running.
This is only relevant if you plan on doing another run on the same cluster afterwards.

In [ ]:
PipelineManager.stop_pipeline(ssh_client)

## 5. Display MultiQC report

### Note: Run the cells below after the multiqc step is done

In [ ]:
# Download the multiqc html file to local
multiqc_report = os.path.join(s3_output_files_address, "{}/{}/multiqc_report.html".format(project_name, workflow))
!aws s3 cp $multiqc_report .

In [ ]:
from IPython.display import IFrame

IFrame(os.path.relpath("multiqc_report.html"), width="100%", height=1000)